In [1]:
import gradio as gr
import tensorflow as tf

/Users/rafaeloliveira/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
import pandas as pd
import tensorflow as tf
import os
import numpy as np

from tensorflow.keras.layers import TextVectorization

df = pd.read_csv('/Users/rafaeloliveira/code/Github Repos/mlops_comment_toxicity/data/train.csv.zip')

X = df['comment_text']
y = df[df.columns[2:]].values

MAX_WORDS = 200_000

vectorizer = TextVectorization(max_tokens=MAX_WORDS,
                               output_sequence_length=1800,
                               output_mode='int')

vectorizer.adapt(X.values)

vectorized_text = vectorizer(X.values)

dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, y))
dataset = dataset.cache()
dataset = dataset.shuffle(160_000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8)

train = dataset.take(int(len(dataset)*.7))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

In [10]:
# Load the Keras model
model_path = "toxicity_2024_10_25.keras"

try:
    model = tf.keras.models.load_model(model_path)
    print("Model loaded successfully.")
except Exception as e:
    print(f"Error loading model: {e}")

Error loading model: Unable to open file (file signature not found)


In [3]:
model = tf.keras.models.load_model('toxicity_2024_10_29.keras')

In [9]:
input_str = vectorizer('hey i am going to kill you')

In [10]:
res = model.predict(np.expand_dims(input_str, 0))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step


In [11]:
res

array([[0.8168662 , 0.00151341, 0.14464559, 0.01468176, 0.19641946,
        0.02815162]], dtype=float32)

In [19]:
df.columns[2:-1]

Index(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult'], dtype='object')

In [17]:
def score_comment(comment):
    vectorized_comment = vectorizer([comment])
    results = model.predict(vectorized_comment)

    text = ''
    for idx, col in enumerate(df.columns[2:-1]):
        text += f'{col}: {results[0][idx]>0.5}\n'

    return text

In [22]:
import gradio as gr
import matplotlib.pyplot as plt
import numpy as np

# Function to generate a score and a plot
def score_comment(comment):
    # Example scoring logic based on comment length
    score = len(comment) % 10  # Score between 0 and 9
    percentage = (score / 10) * 100  # Convert to percentage

    # Create a plot for the score
    fig, ax = plt.subplots(figsize=(6, 4))
    ax.barh(['Score'], [percentage], color='lightcoral')
    ax.set_xlim(0, 100)
    ax.set_xlabel('Percentage (%)', fontsize=12)
    ax.set_title('Score Percentage', fontsize=14, fontweight='bold')

    # Adding grid for better readability
    ax.grid(axis='x', linestyle='--', alpha=0.7)

    # Save the plot to a file and return it
    plt.tight_layout()
    plt.savefig("score_plot.png")
    plt.close(fig)  # Close the figure to avoid display issues in some environments

    return f"Score: {score}/10", "score_plot.png"

# Create the Gradio interface with enhanced visuals
with gr.Blocks() as interface:
    gr.Markdown("<h1 style='text-align: center;'>Comment Scoring App</h1>")
    gr.Markdown("<h3 style='text-align: center;'>Enter a comment to receive a score based on its content.</h3>")

    with gr.Row():
        comment_input = gr.Textbox(lines=2, placeholder='Comment to score', label="Your Comment", elem_id="comment-input")
        submit_button = gr.Button("Score Comment", elem_id="submit-button")

    output_display = gr.Textbox(label="Score", interactive=False, elem_id="score-output")
    plot_output = gr.Image(label="Score Plot", elem_id="plot-output")

    # Define the button action
    submit_button.click(fn=score_comment, inputs=comment_input, outputs=[output_display, plot_output])

# Launch the interface
interface.launch(share=True)

* Running on local URL:  http://127.0.0.1:7863
* Running on public URL: https://ce7dc92e29d42cd753.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
